## O que é RPA?
* **Robotic Process Automation** ou **Automação de Processos Robóticos**
* É uma tecnologia que usa de robôs de software (bots) para automatizar **tarefas repetitivas e baseadas em regras** que normalmente seriam feitas por humanos em sistemas digitais

### Pontos positivos
1. Aumento de produtividade
    * Robôs trabalham 24/7, sem pausa, o que acelera os processos
    <br/>
2. Redução de erros
    * Tarefas repetitivas feitas por humanos são propensas a erros. RPA executa exatamente como programado
    <br/>
3. Economia de custos
    * Após implementado, o RPA reduz custos operacionais com tarefas manuais
    <br/>
4. Integração com sistemas legados
    * Robôs podem operar em sistemas antivos sem necessidades de APIs ou mudanças na estrutura
    <br/>
5. Escalabilidade
    * Você pode aumentar ou reduzir o número de bots conforme a demanda
    <br/>
6. Libera os humanos para tarefas mais estratégicas
    * Com os robôs cuidando do trabalho repetitivo, os colaboradores podem focar em análise e tomada de decisões.


### Pontos negativos
1. Manutenção e atualização constante
    * Mudanças em interfaces ou sistemas podem quebrar o robô, exigindo ajustes rápidos.
    <br/>
2. Custo inicial de implementação
    * Apesar da economia no longo prazo, o setup inicial pode ser caro (licenças, consultorias, desenvolvimento).
    <br/>
3. Limitação a tarefas baseadas em regras
    * Robôs não pensam: só fazem tarefas lógicas e repetitivas. Processos complexos ou que exigem julgamento humano não são ideais.
    <br/>
4. Risco de dependência excessiva
    * Algumas empresas podem automatizar demais e acabar perdendo conhecimento interno sobre os processos.
    <br/>
5. Gestão e governança podem ser desafiadores
    * Sem uma boa estratégia, os bots podem virar uma "caixa-preta" difícil de controlar e auditar.

### Biblioteca utilizada
1. PyAutoGUI
2. instalação: pip3 install PyAutoGUI
3. [Documentação](https://pyautogui.readthedocs.io/en/latest/)
    * Para comandos de imagem pode ser que seja necessário instalar o Pillow
        * instalação: pip3 install pillow

    * Para resolver os problemas de caracteres especiais, será utilizado um macete via pyperclip
4. [Principais comandos PyAutoGUI](https://pyautogui.readthedocs.io/en/latest/quickstart.html)

## Desafio
1. Automatizar a extração de informações de um sistema e envio de um relatório por e-mail
2. O "sistema" escolhido para realizar o desafio, será o Gmail
3. Passo-a-Passo:
    * Parte 1: Entrar no sistema (Entrar no Gmail)
    * Parte 2: Entrar em uma aba específica do sistema onde tem o nosso relatório (Aba Contatos)
    * Parte 3: Exportar o relatório (Exportar Contatos)
    * Parte 4: Pegar o relatório exportado, tratar e pegar as informações que queremos
    * Parte 5: Preencher/Atualizar informações do sistema (No caso, criar um e-mail e enviar)

In [ ]:
# importações
import os
import textwrap
import time
import pyautogui
import pyperclip
import pandas as pd

'''
    pyautogui.write() -> escreve
    pyautogui.click -> clica
    pyautogui.locateOnScreen -> identifica uma imagem na sua tela
    pyautogui.hotkey -> usa atalhos do teclado (combinação de teclas)
    pyautogui.press -> aperta um botão do teclado
    print(pyautogui.KEYBOARD_KEYS)
'''

# CONFIGURAÇÕES INICIAIS
pyautogui.PAUSE = 2 # tempo de 2s entre cada ação
# imagens para pesquisa e acesso ao gmail
IMAGENS = [
    'busca_gmail.png',
    'busca_gmail2.png'
]

# ------------------- FUNÇÕES UTILITÁRIAS -------------------

def start_message():
    try:
        pyautogui.alert("⚠️ A Automação vai começar agora. Por favor não mexa no computador! 🚫🖱️⌨️", timeout=1000)
        pyautogui.press('return')
        log("Iniciando automação..🛠️", "success")
        log("⚠️ A Automação vai começar agora. Por favor não mexa no computador! 🚫🖱️⌨️")
        time.sleep(1)
    except Exception as e:
        log(f"Aviso de início não pôde ser exibido: {e}", "warn")
        time.sleep(3)

def end_message():
    try:
        # pyautogui.alert("✅ Automação finalizada com sucesso! Você já pode usar o computador normalmente.", timeout=2000)
        # pyautogui.press('return')
        log("Automação finalizada com sucesso! Você já pode usar o computador normalmente.", "success")
        time.sleep(1)
    except Exception as e:
        log(f"Aviso final não pode ser exibido: {e}", "warn")

def get_latest_file_csv(folder_path, name_contains='contacts'):
    files = [current_file for current_file in os.listdir(folder_path) if current_file.endswith('.csv') and name_contains in current_file]
    if not files:
        raise FileNotFoundError("⚠️ Nenhum arquivo .csv foi encontrado na pasta 'Downloads'.")
    
    files_paths = [os.path.join(folder_path, current_file) for current_file in files]
    latest_file = max(files_paths, key=os.path.getctime)
    return latest_file

def log(msg, tipo="info"):
    emoji = {"info": "ℹ️", "success": "✅", "error": "❌", "warn": "⚠️", "wait": "⏳"}.get(tipo, "")
    print(f"{emoji} {msg}")

def switching_tabs():
    log('Alternando entre as abas de Contatos e a do Gmail..\n', 'wait')
    pyautogui.hotkey('ctrl', 'tab')

def abrir_navegador():
    ''' ACESSAR O NAVEGADOR ARC USANDO RAYCAST '''
    log("Abrindo navegador Arc via Raycast...\n", "info")
    pyautogui.hotkey('command', 'space')
    time.sleep(1)
    pyautogui.write('Arc')
    pyautogui.press('return')
    # Abrindo uma nova aba
    pyautogui.hotkey('command', 't')


def pesquisar_gmail():
    ''' pesquisar pelo Gmail no navegador '''
    log("Pesquisando Gmail no navegador...\n", "info")
    pyautogui.write('acessar gmail')
    pyautogui.press('return')
    time.sleep(2)


def clicar_centro_imagem(location):
    ''' Efetuando clique centralizado na localização passada por parametro '''
    # calculando o centro da imagem encontrada
    x, y, width, height = location
    center_x = x + width // 2
    center_y = y + height // 2

    log(f"Coordenadas: x={center_x}, y={center_y}\n", "info")
    
    # movendo o mouse gradualmente para a posição
    pyautogui.moveTo(center_x, center_y, duration=0.5)
    time.sleep(0.5)

    # clicando no link
    pyautogui.click()


def encontrar_e_clicar_imagem_gmail():
    ''' LOCALIZA E CLICA NO LINK DA IMAGEM DO GMAIL '''

    log("Procurando pelo link do Gmail...\n", "info")
    time.sleep(1.5)

    location = None

    # varrendo minha lista de imagens e encontrando a localizacao da imagem
    for img in IMAGENS:
        log(f"Tentando localizar: {img}", "info")
        while True:
            try:
                location = pyautogui.locateOnScreen(img, grayscale=True, confidence=0.7)
                log(f"Imagem {img} encontrada com sucesso!", "success")
                clicar_centro_imagem(location)
                return True
            except pyautogui.ImageNotFoundException:
                log(f"\nImagem {img} não foi encontrada!", "warn")
            
    return False


def encontrar_e_clicar_imagem(imagem, mensagem=None, tempo_espera=1, confidence=0.8):

    if mensagem:
        log(mensagem, "info")

    location = None

    try:
        location = pyautogui.locateOnScreen(imagem, grayscale=False, confidence=confidence)
        log(f"Imagem {imagem} encontrada com sucesso!", "success")
        clicar_centro_imagem(location)
        time.sleep(tempo_espera)
        return True
        
    except pyautogui.ImageNotFoundException:
        log(f"Imagem {imagem} não foi encontrada!\n", "error")
            
    return False

def wait_load_page(imagem, timeout=15):
    log(f"Aguardando carregamento da página com a imagem '{imagem}'...", "info")
    start = time.time()
    while time.time() - start < timeout:
        if pyautogui.locateOnScreen(imagem, grayscale=True, confidence=0.6):
            log("Página carregada com sucesso!\n", "success")
            return True
        time.sleep(2)
    log("Tempo de espera esgotado! A Página não carregou...", "error")
    return False

def read_file_csv_and_send_emails():
    csv_path = get_latest_file_csv(r'/Users/brunnomanduca/Downloads')
    df_contacts = pd.read_csv(csv_path)
    df_contacts_clean = df_contacts.dropna(axis=1) # axis=1 -> excluir colunas vazias

    subject_email = 'Teste automação'
    author_email = 'Brunno Manduca'

    for first_name, last_name, email in df_contacts_clean[['First Name', 'Last Name', 'E-mail 1 - Value']].values:

        # corpo do e-mail personalizado para cada contato da lista
        body_email = textwrap.dedent(f"""
        Olá, {first_name},

        Espero que esteja tudo bem com você!

        Estou realizando alguns testes de automação de e-mails utilizando Python (RPA), e este é um envio experimental como parte desse processo.

        Você poderia, por gentileza, me confirmar o recebimento desta mensagem?

        Agradeço antecipadamente pela atenção.

        Atenciosamente,
        {author_email}
        """)

        if last_name:
            encontrar_e_clicar_imagem('button_write_email.png', f"Escrevendo um novo email para {first_name} {last_name}...")
        else:
            encontrar_e_clicar_imagem('button_write_email.png', f"Escrevendo um novo email para {first_name}...")

        # escrevendo os campos necessários para enviar o email
        pyautogui.write(email)
        pyautogui.press('return') # clicando enter para validar o email

        time.sleep(0.5)

        # prosseguindo para o campo assunto
        pyautogui.press('tab')

        # escrevendo o assunto do email
        pyperclip.copy(subject_email)
        pyautogui.hotkey('command', 'v')
        time.sleep(0.5)

        # prosseguindo para o campo de corpo do e-mail
        pyautogui.press('tab')
        time.sleep(0.5)

        # Colando o texto personalizado no corpo do e-mail
        pyperclip.copy(body_email)
        pyautogui.hotkey('command', 'v')
        time.sleep(1)

        # selecionando campo de assinatura
        # Selecionando a opção de assinatura...✏️
        encontrar_e_clicar_imagem('select_signature.png', '')
        time.sleep(0.5)
        # Selecionando a opção de e-mail sem assinatura...
        encontrar_e_clicar_imagem('no_signature.png', '')

        # clicando enviar
        pyautogui.hotkey('command', 'return')
        time.sleep(3)


if __name__ == "__main__":
    start_message()
    time.sleep(1)
    abrir_navegador()
    pesquisar_gmail()
    encontrar_e_clicar_imagem_gmail()
    wait_load_page('logo_gmail_and_search.png')
    encontrar_e_clicar_imagem('menu_google_apps.png',)
    time.sleep(1.5)
    encontrar_e_clicar_imagem('contatos.png')
    wait_load_page('carregar_tela_contatos.png')
    encontrar_e_clicar_imagem('exportar.png', "Clicando na opção 'Exportar'...")
    time.sleep(1)
    encontrar_e_clicar_imagem('selecionar_opcao_contato.png', "Selecionando a opção de Contatos...")
    time.sleep(1)
    encontrar_e_clicar_imagem('selecionar_lista.png', "Selecionando lista de Contatos...")
    time.sleep(1)
    encontrar_e_clicar_imagem('confirmar_exportar.png', "Confirmando a exportação de contatos...")
    time.sleep(2)
    switching_tabs()
    time.sleep(4)
    read_file_csv_and_send_emails()
    time.sleep(2)
    end_message()


## Como descobrir a posição do mouse do local que eu quero

In [1]:
import time
import pyautogui

time.sleep(5)

x, y = pyautogui.position()

print(f"x = {x}, y = {y}")

x = 372, y = 100


In [ ]:
pyautogui.moveTo(161, 139, duration=0.5)
time.sleep(5)
pyautogui.click(161, 139, clicks=2)

In [ ]:
import pandas as pd

df_contacts = pd.read_csv(r'/Users/brunnomanduca/Downloads/contacts.csv')
df_contacts_clean = df_contacts.dropna(axis=1)

display(df_contacts_clean)

for first_name in df_contacts_clean['First Name']:
        print(f"Escrevendo um novo email para {first_name}...")

for first_name, last_name, email in df_contacts_clean[['First Name', 'Last Name', 'E-mail 1 - Value']].values:
        print(f"Escrevendo um novo email para {first_name} {last_name}, com o email: {email}...")

## Processo trocar entre abas

In [ ]:
import pyautogui

pyautogui.moveTo(452, 329, duration=0.5)
pyautogui.click(clicks=2)
time.sleep(2)

while not pyautogui.locateOnScreen('chatgpt.png', grayscale=False, confidence=0.3):
    pyautogui.hotkey('command', 'shift', 'tab')
print('achei a tela do Chatpgpt')